## Description of Variables

A dataset of heartrates is used for this project. Each row represents a single measurement. Columns labeled as T1 from T80 are the time steps on the timeline (there are 80 time steps, each time step has only one measurement). 

The last column is the target variable. It shows the label (category) of the measurement as follows:<br>
0 = Normal<br>
1 = Supraventricular premature beat<br>
2 = Premature ventricular contraction<br>
3 = Fusion of ventricular and normal beat<br>
4 = Unclassifiable beat

## Goal

Use the data set **hearbeat_cleaned.csv** to predict the column called **Target**. The input variables are columns labeled as **T1 to T80**. 

# Read and Prepare the Data

In [1]:
# Import libraries

import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

# Other Libararies
from tensorflow.keras.callbacks import EarlyStopping

# Random Seed
np.random.seed(42)
tf.random.set_seed(42)

In [2]:
# Read the dataset

data = pd.read_csv(r'C:\Users\Scott\Downloads\heartbeat_cleaned.csv')

In [3]:
data.shape

(7960, 81)

In [4]:
data.head()

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,...,T72,T73,T74,T75,T76,T77,T78,T79,T80,Target
0,0.987,0.892,0.461,0.1130,0.1490,0.1900,0.1650,0.1620,0.1470,0.1380,...,0.1970,0.1970,0.1960,0.2030,0.201,0.1990,0.2010,0.205,0.2080,0
1,1.000,0.918,0.621,0.1330,0.1050,0.1250,0.1170,0.0898,0.0703,0.0781,...,0.1950,0.1910,0.1520,0.1720,0.207,0.2110,0.2070,0.207,0.1720,0
2,1.000,0.751,0.143,0.1040,0.0961,0.0519,0.0442,0.0416,0.0364,0.0857,...,0.2260,0.2420,0.2440,0.2860,0.468,0.8160,0.9770,0.452,0.0519,0
3,1.000,0.740,0.235,0.0464,0.0722,0.0567,0.0103,0.0155,0.0284,0.0155,...,0.0851,0.0747,0.0515,0.0593,0.067,0.0361,0.1210,0.451,0.8690,0
4,1.000,0.833,0.309,0.0191,0.1010,0.1200,0.1040,0.0874,0.0765,0.0765,...,0.2050,0.4210,0.8030,0.9510,0.467,0.0000,0.0519,0.082,0.0628,0


In [5]:
# Separate the target variable

y = data['Target']
x = data.drop('Target', axis=1)

## Split the data

In [6]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3)

## Data Transformation

In [7]:
# Convert the target variable to integer type
train_y = np.array(train_y)
test_y = np.array(test_y)

train_y = train_y.astype(np.int32)
test_y = test_y.astype(np.int32)

In [8]:
# Take a look at what the first 10 values look like of the target variable
train_y[0:10]

array([1, 0, 2, 0, 2, 2, 0, 0, 4, 4])

In [9]:
# Convert the input variables to float data types
train_x= np.array(train_x)
test_x= np.array(test_x)

train_x = train_x.astype(np.float32)
test_x = test_x.astype(np.float32)

In [10]:
train_x

array([[1.    , 0.818 , 0.133 , ..., 0.354 , 0.354 , 0.387 ],
       [1.    , 0.849 , 0.166 , ..., 0.108 , 0.0811, 0.0695],
       [0.    , 0.0335, 0.163 , ..., 0.62  , 0.624 , 0.606 ],
       ...,
       [1.    , 0.98  , 0.573 , ..., 0.057 , 0.038 , 0.0418],
       [0.564 , 0.512 , 0.468 , ..., 0.357 , 0.348 , 0.354 ],
       [0.799 , 0.683 , 0.564 , ..., 0.265 , 0.265 , 0.247 ]],
      dtype=float32)

In [11]:
# Convert the dataset to 3-D

train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], 1))
test_x = np.reshape(test_x, (test_x.shape[0], test_x.shape[1], 1))

In [12]:
train_x.shape, train_y.shape

((5572, 80, 1), (5572,))

In [13]:
train_x

array([[[1.    ],
        [0.818 ],
        [0.133 ],
        ...,
        [0.354 ],
        [0.354 ],
        [0.387 ]],

       [[1.    ],
        [0.849 ],
        [0.166 ],
        ...,
        [0.108 ],
        [0.0811],
        [0.0695]],

       [[0.    ],
        [0.0335],
        [0.163 ],
        ...,
        [0.62  ],
        [0.624 ],
        [0.606 ]],

       ...,

       [[1.    ],
        [0.98  ],
        [0.573 ],
        ...,
        [0.057 ],
        [0.038 ],
        [0.0418]],

       [[0.564 ],
        [0.512 ],
        [0.468 ],
        ...,
        [0.357 ],
        [0.348 ],
        [0.354 ]],

       [[0.799 ],
        [0.683 ],
        [0.564 ],
        ...,
        [0.265 ],
        [0.265 ],
        [0.247 ]]], dtype=float32)

# Find the baseline

In [14]:
data['Target'].value_counts()/len(data)

0    0.582035
4    0.198995
2    0.155402
1    0.055905
3    0.007663
Name: Target, dtype: float64

# Build a cross-sectional shallow model using Keras (with only one hidden layer)

In [15]:
# Using 80 because we have 80 columns and 1 because we have 1 column as target variable

model = keras.models.Sequential([
    
    keras.layers.Flatten(input_shape=[80, 1]),
    keras.layers.Dense(80, activation='relu'),
    keras.layers.Dense(5, activation='softmax')
    
])

In [16]:
# Make sure to add our random seeds for both numpy and tensorflow
np.random.seed(42)
tf.random.set_seed(42)

# Configure the optimizer
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

# Fit the model
history = model.fit(train_x, train_y, epochs=50,
                    validation_data=(test_x, test_y))

Epoch 1/50
175/175 [==============================] - 3s 8ms/step - loss: 0.6766 - accuracy: 0.7740 - val_loss: 0.6168 - val_accuracy: 0.8291
Epoch 2/50
175/175 [==============================] - 1s 5ms/step - loss: 0.4439 - accuracy: 0.8580 - val_loss: 0.4514 - val_accuracy: 0.8539
Epoch 3/50
175/175 [==============================] - 1s 5ms/step - loss: 0.3755 - accuracy: 0.8878 - val_loss: 0.6498 - val_accuracy: 0.7563
Epoch 4/50
175/175 [==============================] - 1s 5ms/step - loss: 0.3435 - accuracy: 0.8982 - val_loss: 0.3440 - val_accuracy: 0.9012
Epoch 5/50
175/175 [==============================] - 1s 5ms/step - loss: 0.3197 - accuracy: 0.9040 - val_loss: 0.3155 - val_accuracy: 0.9024
Epoch 6/50
175/175 [==============================] - 1s 5ms/step - loss: 0.3034 - accuracy: 0.9092 - val_loss: 0.4579 - val_accuracy: 0.8580
Epoch 7/50
175/175 [==============================] - 1s 5ms/step - loss: 0.2842 - accuracy: 0.9169 - val_loss: 0.3691 - val_accuracy: 0.8848
Epoch 

In [17]:
# Evaluate the model
scores = model.evaluate(test_x, test_y, verbose=0)

# Extract the accuracy from model.evaluate
print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.22
accuracy: 93.89%


# Build a cross-sectional deep model using Keras (with two or more hidden layers)

In [18]:
# Create early stopping
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

callback = [earlystop]

In [19]:
# Create the input shapes (80 for input columns and 1 for target column)
n_steps = 80
n_inputs = 1

# Create the model
model = keras.models.Sequential([
    keras.layers.SimpleRNN(50, return_sequences=True, input_shape=[n_steps, n_inputs] ),
    keras.layers.SimpleRNN(35, return_sequences=True),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(5), 
    keras.layers.Dense(5, activation='softmax')
])


In [20]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/20
175/175 [==============================] - 30s 140ms/step - loss: 0.9774 - accuracy: 0.7032 - val_loss: 0.8515 - val_accuracy: 0.7450
Epoch 2/20
175/175 [==============================] - 23s 131ms/step - loss: 0.7875 - accuracy: 0.7487 - val_loss: 0.6624 - val_accuracy: 0.7889
Epoch 3/20
175/175 [==============================] - 23s 132ms/step - loss: 0.6490 - accuracy: 0.8022 - val_loss: 0.8379 - val_accuracy: 0.7127
Epoch 4/20
175/175 [==============================] - 23s 132ms/step - loss: 0.6280 - accuracy: 0.8110 - val_loss: 0.5696 - val_accuracy: 0.8266
Epoch 5/20
175/175 [==============================] - 23s 131ms/step - loss: 0.5913 - accuracy: 0.8139 - val_loss: 0.5603 - val_accuracy: 0.8245
Epoch 6/20
175/175 [==============================] - 23s 132ms/step - loss: 0.4972 - accuracy: 0.8591 - val_loss: 0.4940 - val_accuracy: 0.8446
Epoch 7/20
175/175 [==============================] - 23s 133ms/step - loss: 0.5166 - accuracy: 0.8500 - val_loss: 0.5020 - val_ac

In [21]:
# Evaluate the model
scores = model.evaluate(test_x, test_y, verbose=0)

# Extract the accuracy from model.evaluate
print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.43
accuracy: 88.65%


# Build a sequential shallow LSTM Model (with only one LSTM layer)

In [22]:
# Create the input shapes (80 for input columns and 1 for target column)
n_steps = 80
n_inputs = 1

model = keras.models.Sequential([
    
    keras.layers.LSTM(5, activation='softmax' , input_shape=[n_steps, n_inputs])
])

In [23]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.001)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/20
175/175 [==============================] - 14s 66ms/step - loss: 1.4588 - accuracy: 0.5770 - val_loss: 1.3335 - val_accuracy: 0.5930
Epoch 2/20
175/175 [==============================] - 11s 61ms/step - loss: 1.2820 - accuracy: 0.5774 - val_loss: 1.2165 - val_accuracy: 0.5930
Epoch 3/20
175/175 [==============================] - 11s 61ms/step - loss: 1.2138 - accuracy: 0.5774 - val_loss: 1.1689 - val_accuracy: 0.5930
Epoch 4/20
175/175 [==============================] - 11s 62ms/step - loss: 1.1742 - accuracy: 0.5774 - val_loss: 1.1358 - val_accuracy: 0.5930
Epoch 5/20
175/175 [==============================] - 11s 62ms/step - loss: 1.1439 - accuracy: 0.5774 - val_loss: 1.1116 - val_accuracy: 0.5930
Epoch 6/20
175/175 [==============================] - 10s 59ms/step - loss: 1.1192 - accuracy: 0.5772 - val_loss: 1.0873 - val_accuracy: 0.5925
Epoch 7/20
175/175 [==============================] - 11s 61ms/step - loss: 1.0972 - accuracy: 0.5718 - val_loss: 1.0687 - val_accuracy:

In [24]:
# Evaluate the model
scores = model.evaluate(test_x, test_y, verbose=0)

# Extract the accuracy from model.evaluate
print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 1.03
accuracy: 60.89%


# Build a sequential deep LSTM Model (with only two LSTM layers)

In [25]:
n_steps = 80
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.LSTM(15, return_sequences=True),
    keras.layers.LSTM(10),
    keras.layers.Dense(5, activation='softmax')
]) #

In [26]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/20
175/175 [==============================] - 48s 206ms/step - loss: 1.1413 - accuracy: 0.5723 - val_loss: 1.1072 - val_accuracy: 0.5930
Epoch 2/20
175/175 [==============================] - 33s 188ms/step - loss: 1.1309 - accuracy: 0.5765 - val_loss: 1.0841 - val_accuracy: 0.5930
Epoch 3/20
175/175 [==============================] - 33s 187ms/step - loss: 1.1198 - accuracy: 0.5808 - val_loss: 1.1188 - val_accuracy: 0.5930
Epoch 4/20
175/175 [==============================] - 32s 185ms/step - loss: 1.1216 - accuracy: 0.5793 - val_loss: 1.1122 - val_accuracy: 0.5930
Epoch 5/20
175/175 [==============================] - 33s 188ms/step - loss: 1.1363 - accuracy: 0.5774 - val_loss: 1.1310 - val_accuracy: 0.5930
Epoch 6/20
175/175 [==============================] - 32s 186ms/step - loss: 1.1358 - accuracy: 0.5774 - val_loss: 1.1092 - val_accuracy: 0.5930
Epoch 7/20
175/175 [==============================] - 33s 188ms/step - loss: 1.1351 - accuracy: 0.5774 - val_loss: 1.1086 - val_ac

In [27]:
# Evaluate the model
scores = model.evaluate(test_x, test_y, verbose=0)

# Extract the accuracy from model.evaluate
print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 1.11
accuracy: 59.30%


# Build a sequential shallow GRU Model (with only one GRU layer)

In [28]:
n_steps = 80
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(5, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(5, activation='softmax')
])

In [29]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.001)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/20
175/175 [==============================] - 18s 78ms/step - loss: 1.3909 - accuracy: 0.5384 - val_loss: 1.1463 - val_accuracy: 0.5930
Epoch 2/20
175/175 [==============================] - 13s 72ms/step - loss: 1.1473 - accuracy: 0.5774 - val_loss: 1.1148 - val_accuracy: 0.5930
Epoch 3/20
175/175 [==============================] - 12s 71ms/step - loss: 1.1257 - accuracy: 0.5774 - val_loss: 1.0945 - val_accuracy: 0.5930
Epoch 4/20
175/175 [==============================] - 13s 72ms/step - loss: 1.1071 - accuracy: 0.5770 - val_loss: 1.0739 - val_accuracy: 0.5921
Epoch 5/20
175/175 [==============================] - 13s 72ms/step - loss: 1.0874 - accuracy: 0.5721 - val_loss: 1.0678 - val_accuracy: 0.6013
Epoch 6/20
175/175 [==============================] - 12s 71ms/step - loss: 1.0755 - accuracy: 0.5791 - val_loss: 1.0475 - val_accuracy: 0.5942
Epoch 7/20
175/175 [==============================] - 12s 71ms/step - loss: 1.0681 - accuracy: 0.5845 - val_loss: 1.0423 - val_accuracy:

In [30]:
# Evaluate the model
scores = model.evaluate(test_x, test_y, verbose=0)

# Extract the accuracy from model.evaluate
print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.93
accuracy: 61.39%


# Build a sequential deep GRU Model (with only two GRU layers)

In [31]:
n_steps = 80
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(5, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.GRU(5, return_sequences=True),
    keras.layers.GRU(5, return_sequences=True),
    keras.layers.GRU(5),
    keras.layers.Dense(5, activation='softmax')
])

In [32]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/20
175/175 [==============================] - 69s 298ms/step - loss: 1.1460 - accuracy: 0.5774 - val_loss: 1.0623 - val_accuracy: 0.5930
Epoch 2/20
175/175 [==============================] - 49s 278ms/step - loss: 0.9957 - accuracy: 0.6068 - val_loss: 0.9304 - val_accuracy: 0.5804
Epoch 3/20
175/175 [==============================] - 49s 280ms/step - loss: 0.7608 - accuracy: 0.7227 - val_loss: 0.6970 - val_accuracy: 0.7688
Epoch 4/20
175/175 [==============================] - 49s 278ms/step - loss: 0.6087 - accuracy: 0.8065 - val_loss: 0.8240 - val_accuracy: 0.7123
Epoch 5/20
175/175 [==============================] - 49s 279ms/step - loss: 0.5410 - accuracy: 0.8372 - val_loss: 0.5133 - val_accuracy: 0.8405
Epoch 6/20
175/175 [==============================] - 49s 279ms/step - loss: 0.5091 - accuracy: 0.8496 - val_loss: 0.5021 - val_accuracy: 0.8501
Epoch 7/20
175/175 [==============================] - 49s 278ms/step - loss: 0.4792 - accuracy: 0.8546 - val_loss: 1.1088 - val_ac

In [33]:
# Evaluate the model
scores = model.evaluate(test_x, test_y, verbose=0)

# Extract the accuracy from model.evaluate
print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.44
accuracy: 87.56%


# Discussion

## List the test values of each model you built

## Which model performs the best and why?

## How does it compare to baseline? (0.5 points)